In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie
    
    
    


In [8]:
# 2 Add the function from our homework that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_file, kaggle_file, ratings_file):
#Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)
    
# Open and read the Wikipedia data JSON file but don't transform into a dataframe until cleaning it up.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file)

        
# 3. Write a list comprehension to filter out TV shows.
        wiki_movies = [movie for movie in wiki_movies_raw
                   if ('Director' in movie or 'Directed by' in movie)
                       and 'imdb_link' in movie
                       and 'No. of episodes' not in movie]

        
# 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    
# 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
    
#     print(wiki_movies_df)
#     print(kaggle_metadata)
#     print(ratings)
    
 
    
# 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    
        # Get the unique identifier for each movie by extracting the IMDB ID from the IMDB link column using regex 
        # Then add the IMDB ID column to the dataframe
        # With this column,we can drop duplicates

    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})') 
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as uhoh:
        print(uhoh)
 
 #7. Write a list comprehension to remove mostly null columns from the wiki_movies_df DataFrame.  We're keeping columns with
     #less than 90% null values
   
        wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
        wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
        print(wiki_movies_df.isnull().sum())
    
 # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()

    return wiki_movies_df, kaggle_metadata, ratings

In [9]:
file_dir = 'C://Users/mill2313/Movies_ETL/Resources/'
wiki_file = f'{file_dir}/wikipedia-movies.json'
kaggle_file = f'{file_dir}/movies_metadata.csv'
ratings_file= f'{file_dir}/ratings.csv'


extract_transform_load(wiki_file,kaggle_file,ratings_file)

(                                                    url  year  \
 0     https://en.wikipedia.org/wiki/The_Adventures_o...  1990   
 1     https://en.wikipedia.org/wiki/After_Dark,_My_S...  1990   
 2      https://en.wikipedia.org/wiki/Air_America_(film)  1990   
 3       https://en.wikipedia.org/wiki/Alice_(1990_film)  1990   
 4         https://en.wikipedia.org/wiki/Almost_an_Angel  1990   
 ...                                                 ...   ...   
 7071    https://en.wikipedia.org/wiki/Holmes_%26_Watson  2018   
 7072     https://en.wikipedia.org/wiki/Vice_(2018_film)  2018   
 7073  https://en.wikipedia.org/wiki/On_the_Basis_of_Sex  2018   
 7074  https://en.wikipedia.org/wiki/Destroyer_(2018_...  2018   
 7075  https://en.wikipedia.org/wiki/Black_Mirror:_Ba...  2018   
 
                                   imdb_link                            title  \
 0     https://www.imdb.com/title/tt0098987/  The Adventures of Ford Fairlane   
 1     https://www.imdb.com/title/tt0098994/

In [ ]:

    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
   
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    

    # 12. Add the parse_dollars function.
    
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.

    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
   
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    # 17. Create the path to your file directory and variables for the three files.

    # Wikipedia data

    # Kaggle metadata

    # MovieLens rating data.

    # 18. Set the three variables equal to the function created in D1.

    # 19. Set the wiki_movies_df equal to the wiki_file variable. 

    # 20. Check that the wiki_movies_df DataFrame looks like this. 


    # 21. Check that wiki_movies_df DataFrame columns are correct. 